In [9]:
import pymongo
from pymongo import MongoClient
import pandas as pd
from pandas import DataFrame
from bson import ObjectId
from pandas import json_normalize
# from pandas.io.json import json_normalize
print ("pymongo version:", pymongo.version)

pd.set_option('display.max_columns', 1010)
pd.set_option('display.max_rows', 1010)

pymongo version: 3.10.1


### Check status mongodb
* sudo service mongodb status
* sudo service mongodb start

In [10]:
mongo_client = MongoClient('localhost', 27017)
host_info = mongo_client['HOST']
print ("\nhost:", host_info)


host: Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'HOST')


### List databases

In [11]:
print(mongo_client.list_database_names())

['admin', 'legalthings', 'local']


In [12]:
legalthings_db = mongo_client["legalthings"]

### Show collections in database

In [13]:
all_collections = legalthings_db.list_collection_names ()

### Explore query possibilities

#### One '_id' can have multiple persons, search on objectid

In [31]:
legalthings_db['processes_18'].find({})[0]

{'_id': ObjectId('5e0f3964526253513f8b4567'),
 'title': 'BV Oprichten',
 'name': 'Jan George | n.t.b',
 'scenario': {'name': 'BV Oprichten (funnel)',
  'image': None,
  'description': None,
  'alias': [{'key': 'spaar-bv', 'value': 'Spaar BV Oprichten'},
   {'key': 'normal-bv', 'value': 'BV Oprichten'},
   {'key': 'holding', 'value': 'Holding Oprichten'},
   {'key': 'inbreng', 'value': 'BV Oprichten met Inbreng'},
   {'key': 'fusie', 'value': 'Aandelen-/Bedrijfsfusie'},
   {'key': 'zorg', 'value': 'Zorg BV Oprichten'},
   {'key': 'stichting', 'value': 'Stichting Oprichten'}],
  'title': 'BV Oprichten',
  'actors': [{'title': 'Cliënt',
    'requirement': None,
    'id': None,
    'organization': None,
    'name': None,
    'absent': False,
    'key': 'user'},
   {'title': 'Notaris',
    'requirement': None,
    'id': {'<ref>': 'global.config.legalflow.workflows.general.notary.id'},
    'organization': {'id': None,
     'name': None,
     'type': None,
     '<ref>': 'global.config.legalfl

In [ ]:
cursor = legalthings_db['processes_18'].find({"_id": ObjectId("5e0f7522643b53c4248b4568")})

In [ ]:
cursor[0]['private_data']['meta_data']['ah']

#### Check if column exists

In [ ]:
cursor = legalthings_db['processes_18'].find({"private_data.meta_data.ah": {"$exists": True}})

In [ ]:
cursor.count()

#### Normalize on nested JSON

In [ ]:
cursor = legalthings_db['processes_18'].find({"private_data.meta_data.ah": {"$exists": True}})

In [ ]:
df = json_normalize(cursor, [['private_data', 'meta_data', 'ah']], ['_id', 'name'])

In [ ]:
len(df['_id'].unique()), df.shape

#### Find Documents using Logical Conditions
* https://specify.io/how-tos/find-documents-in-mongodb-using-the-mongo-shell

In [24]:
cursor = legalthings_db['processes_18'].find({
    "$and": [
        {"current.title": "Cliënt rondt de betaling af"},
        {"private_data.meta_data.ah": {"$exists": True}}
    ]
})

In [25]:
cursor.count()

/mnt/c/projects/riskmodel/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2

In [21]:
cursor = legalthings_db['processes_18'].find({
    "$or": [
        {"_id": ObjectId("5e0f3964526253513f8b4567")},
        {"_id": ObjectId("5e0f39b97873f1bf54cb660b")}
  ]
})

In [22]:
cursor.count()

/mnt/c/projects/riskmodel/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2

In [42]:
cursor = legalthings_db['processes_18'].find({
    "$and": [
        {
            "$or": [{"_id": ObjectId("5e0f3964526253513f8b4567")},
                    {"_id": ObjectId("5e0f396fda6fdf1e5860a9e0")}]
        },
        {
            "$or": [{"name": "Jan George | n.t.b"},
                    {"name": "Michael Nieveld | n.t.b"}]
        }
    ]
})

In [43]:
cursor.count()

/mnt/c/projects/riskmodel/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2

### Make dataframe of collection

In [ ]:
df = DataFrame(list(legalthings_db.incorporation_processes.find({})))

In [ ]:
df.shape

In [ ]:
df.head(n=2)

#### Normalize JSON

However the subdocuments will just appear as JSON inside the subdocument cell. If you want to flatten objects so that subdocument properties are shown as individual cells you can use json_normalize without any parameters.

In [ ]:
datapoints = list(legalthings_db.incorporation_processes.find({}))

In [ ]:
df = json_normalize(datapoints)

### First exploration of the data
* Important note: collection is still small subset of the total collection, otherwise we run into memory problems
* In a later stadium, most important data is extracted the total collection

In [ ]:
df.shape, len(df['_id'].unique()), len(df['name'].unique()), df['title'].unique(), df['current.title'].unique()

Drop columns with all missings

In [ ]:
df = df.dropna(axis=1, how='all')

In [ ]:
df.shape

Get duplicate company names

In [ ]:
df[df['name'].duplicated(keep=False)].sort_values('name').head()

Get companies with status 'Cliënt rond de betaling af' --> probably shows whole funnel

In [ ]:
companies_funnel_all = (
    df[df['current.title'].isin(['Cliënt rond de betaling af', 'Cliënt rondt de betaling af'])]
    ['name']
    .unique()
)

len(companies_funnel_all)

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')

#### Still some nested data --> show 1 collection to explore JSON structure

In [ ]:
legalthings_db.incorporation_processes.find_one()

Example unnest 'scenario.alias' and add some extra information
* https://stackoverflow.com/questions/47242845/pandas-io-json-json-normalize-with-very-nested-json

In [ ]:
json_normalize(datapoints, [['scenario', 'alias']], ['_id', 'title','name']).head()

#### Unnest company with probably a lot of information within nested fields

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')

In [ ]:
filter_query = { "name": "MEA" }

datapoint = list(legalthings_db.incorporation_processes.find(filter_query))

#### private_data.meta_data.kvk.type_toelichting

In [ ]:
df[df['name'] == companies_funnel_all[0]].dropna(axis=1, how='all')['private_data.meta_data.kvk.type_toelichting']

#### private_data.meta_data.ah --> Nice, NAW information

In [ ]:
json_normalize(datapoint, [['private_data', 'meta_data', 'ah']], ['_id', 'title','name'])

#### private_data.assets.invoice.versions

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'versions']], ['_id', 'title','name'])

#### private_data.assets.invoice.created.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'created', 'user', 'employment']])

#### private_data.assets.invoice.last_updated.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'assets', 'invoice', 'last_updated', 'user', 'employment']])

#### private_data.products

In [ ]:
json_normalize(datapoint, [['private_data', 'products']])

#### private_data.payment.products

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'products']])

#### private_data.payment.user.authz_groups

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'user', 'authz_groups']])

#### private_data.payment.user.employment

In [ ]:
json_normalize(datapoint, [['private_data', 'payment', 'user', 'employment']])